In [171]:
import sys
from PySide6 import QtWidgets
from PySide6 import QtCore
from PySide6 import QtGui
from PySide6.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QPushButton
from PySide6.QtSerialPort import QSerialPort, QSerialPortInfo
from PySide6.QtCore import QIODevice, QByteArray
from PySide6.QtCore import QTime, QTimer, Slot
import numpy as np
import pandas as pd
from time import *
import concurrent.futures
import os
import cv2
import traceback
import ctypes
myappid = 'nil.npm.pyqt.2' # arbitrary string
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)

from PySide6.QtGui import *
from PySide6.QtWidgets import *
from PySide6.QtCore import *

from importlib import reload

import ui_main
reload(ui_main)
from ui_main import *

import ui_adjust
reload(ui_adjust)
from ui_adjust import *

from scaleBar import scaleBar

videoFormats = [
    "avi",
    "mp4",
    "mov",
    "mkv",
    "wmv",
    "flv",
    "mpeg",
    "mpg",
]

data=None

def getMaxThreads():
    # Get the number of available CPUs
    numCPUs = os.cpu_count()
    
    # Default maximum threads in ThreadPoolExecutor
    maxThreads = numCPUs * 5 if numCPUs else 1
    
    return maxThreads

def resizeVideos(rows, cols, width, height, layout): # resize all videos to fit
    videos=[]
    for row in range(rows): 
        for col in range(cols):
            widget = layout.itemAtPosition(row, col)
            if widget is not None:
                video = widget.widget()
                video.setMaximumSize(width, height)
                video.setIconSize(QtCore.QSize(width, height))
                videos.append(video)
    return videos

def clearVideos(layout):
    for i in reversed(range(layout.count())):
        widget = layout.itemAt(i).widget()
        if widget is not None:
            widget.setParent(None)

def cv2FrameToPixmap(frame):
    # Convert BGR (OpenCV format) to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert the image to a QImage
    height, width, channel = frame.shape
    bytesPerLine = 3 * width
    qImage = QImage(frame.data, width, height, bytesPerLine, QImage.Format_RGB888)

    # Convert QImage to QPixmap
    pixmap = QPixmap.fromImage(qImage)
    return pixmap

class WorkerSignals(QObject): # Source: https://www.pythonguis.com/tutorials/multithreading-pyside6-applications-qthreadpool/
    '''
    Defines the signals available from a running worker thread.
    
    Supported signals are:
    
    error
        tuple (exctype, value, traceback.format_exc() )

    result
        object data returned from processing, anything
    '''

    error = Signal(tuple)
    result = Signal(object)
    
class VideoWorker(QRunnable): # Source: https://www.pythonguis.com/tutorials/multithreading-pyside6-applications-qthreadpool/
    def __init__(self, mutex, fn, *args, **kwargs):
        super(VideoWorker, self).__init__()

        # Store constructor arguments (re-used for processing)
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()
        self.mutex = mutex

        # Add the callback to our kwargs
        self.kwargs['resultCallback'] = self.signals.result

        # run flag
        self._running = True

    def stop(self):
        print('STOP')
        self.running = False # stop flag
    
    @Slot()
    def run(self):
        '''
        Initialise the runner function with passed args, kwargs.
        '''
        if self._running:
            # Retrieve args/kwargs here; and fire processing using them
            try:
                result = self.fn(*self.args, **self.kwargs)
            except:
                traceback.print_exc()
                exctype, value = sys.exc_info()[:2]
                self.signals.error.emit((exctype, value, traceback.format_exc()))
            else:
                self.mutex.lock()
                self.signals.result.emit(result)  # Return the result of the processing
                self.mutex.unlock()       

class Adjustments(QDialog, Ui_Adjust): # adjustments popup
    sendBlur = Signal(tuple)
    sendDilate = Signal(tuple)
    
    def __init__(self, parent=None):
        QDialog.__init__(self, parent)
        self.setupUi(self)
        self.setWindowTitle("Adjustments")
        self.setWindowIcon(QtGui.QIcon('adjIcon.png'))

        self.blurSlider.valueChanged.connect(self.blurValue.setValue)
        self.blurValue.valueChanged.connect(self.blurSlider.setValue)
        self.blurToggle.toggled.connect(self.blurValue.setEnabled)
        self.blurToggle.toggled.connect(self.blurSlider.setEnabled)
        self.blurToggle.toggled.connect(self.blur)
        self.dilationToggle.toggled.connect(self.dilationSlider.setEnabled)
        self.dilationToggle.toggled.connect(self.dilate)
        self.dilationValue.valueChanged.connect(self.dilationSlider.setValue)
        self.dilationSlider.valueChanged.connect(self.dilationValue.setValue)
        self.dilationToggle.toggled.connect(self.dilationValue.setEnabled)
        
        self.blurValue.valueChanged.connect(self.blur)
        self.dilationValue.valueChanged.connect(self.dilate)

    def blur(self):
        self.sendBlur.emit((self.blurValue.value(), self.blurToggle.isChecked()))

    def dilate(self):
        self.sendDilate.emit((self.dilationValue.value(), self.dilationToggle.isChecked()))

class MainWindow(QtWidgets.QMainWindow, Ui_MainWindow): # main window
    def __init__(self):
        super(MainWindow, self).__init__()
        self.setupUi(self)
        self.mutex = QMutex() # locking threads
        
        # setup threads
        self.threadPool = QtCore.QThreadPool()
        self.threadPool.setMaxThreadCount(getMaxThreads())
        self.videoConnections = []
        
        # set icon and title
        self.setWindowIcon(QtGui.QIcon('icon.png'))
        self.setWindowTitle("NPM Video Analyzer")

        self.videoFiles = [] # store file paths
        
        self.actionOpen_Video.triggered.connect(self.openFile)  # Menubar "Open"
        self.actionSave.triggered.connect(self.exportVideo) # Menubar "Export"
        self.actionClear.triggered.connect(self.onClear) # Menubar "Clear"
        self.actionScale_Bar.triggered.connect(self.onScaleBar) # Menubar "Scale Bar"
        self.actionAdjustments.triggered.connect(self.onAdjustments) # Menubar "Adjustments"
        self.actionData.triggered.connect(self.showData) # Menubar "Data"
        self.actionMain.triggered.connect(self.showMain) # Menubar "Main"
        self.actionPoints_2.triggered.connect(self.onPoints) # Menubar "Points"
        self.actionBounding_Boxes_2.triggered.connect(self.onBB) # Menubar "Bounding Boxes"
        self.scaleBarOn = 1

        # connect sliders
        self.contourValue.valueChanged.connect(self.contourFunction)
        self.contrastValue.valueChanged.connect(self.contrastFunction)
        self.thresholdValue.valueChanged.connect(self.thresholdFunction)
        self.frameDiffValue.valueChanged.connect(self.onFrameDifferencing)
        self.frameDiffToggle.toggled.connect(self.onFrameDifferencing)
        self.subBackValue.valueChanged.connect(self.subtractBackgroundFunction)
        self.subBackToggle.toggled.connect(self.subtractBackgroundFunction)
        self.subBackMethod.currentIndexChanged.connect(self.subtractBackgroundFunction)
        self.consecutiveFramesValue.valueChanged.connect(self.onConsecutiveFramesValue)
        self.record.toggled.connect(self.onRecord)
        self.contDetectValue.valueChanged.connect(self.onPersistence)

        # initialize values
        self.thresholdVal = 0
        self.contrastVal = 1
        self.contourVal = 1

        # initialize video display
        self.videos = []
        self.testButton.clicked.connect(self.addVideos)
        self.testButton.hide() # test button to add blank videos
        self.emptyVideo = QPixmap(u"background.jpg") # blank video background
        self.emptyTruth = True
        self.numColumns = 2
        self.dim = [640, 480]
        self.numVideos = 1
        self.countPlaceholder=0
        self.COUNTADDVIDEO = 0 # testing  count variables
        self.COUNTCAPTURE = 0
        self.COUNTPARSE = 0
        self._running = True
        self.backSubsMOG2={} # background subtraction
        self.backSubsKNN={}
        self.subBackVal = 0
        self.blurVal=0
        self.blurToggle=False
        self.dilateVal=0
        self.dilateToggle=False
        self.consecutiveFrames=12 # for frame diff
        self.frameDifVal=30
        self.frameDiffValue.setValue(30)
        self.resetFrames=False
        self.bb = True # bounding boxes
        self.points = False # points
        self.persistence = 5

        # set empty placeholder video
        self.video_1.setIcon(QtGui.QIcon(u"background.jpg"))
        self.video_1.setMaximumSize(self.dim[0], self.dim[1])
        self.video_1.setIconSize(QtCore.QSize(self.dim[0]*.99, self.dim[1]*.99))
        self.video_1.setCheckable(True)

        # check video states
        # self.timer = QTimer()
        # self.timer.timeout.connect(self.checkCounts)
        # self.timer.start(333)

        # data collection
        self.data={}
        self.dataTemp={}
        
        # show main
        self.showMain()

    def onPersistence(self):
        self.persistence = 1+5*self.contDetectValue.value()/100
    
    def onRecord(self):
        global data
        data = pd.DataFrame(self.data)
        print(data)
    
    def onPoints(self):
        self.actionBounding_Boxes_2.setChecked(False)
        self.bb = False
        self.points = True
    
    def onBB(self):
        self.actionPoints_2.setChecked(False)
        self.bb = True
        self.points = False

    def showMain(self):
        self.tabs.setCurrentWidget(self.mainView)

    def showData(self):
        self.tabs.setCurrentWidget(self.dataView)

    def onAdjustments(self):
        AdjustmentsPopup = Adjustments(self)
        AdjustmentsPopup.sendBlur.connect(self.onBlur)
        AdjustmentsPopup.sendDilate.connect(self.onDilate)
        AdjustmentsPopup.setModal(False)  # Make the dialog non-modal
        AdjustmentsPopup.show()  # Use show() to display the dialog
        
    def onBlur(self, data):
        def nearestOdd(n):
            # If n is odd, return it, otherwise add 1 to make it odd
            return n if n % 2 == 1 else n + 1
        self.blurVal=nearestOdd(int(31*data[0]/100)) # 31 is the reccomended max given by ChatGPT
        self.blurToggle=data[1]
        if self.blurToggle:
            self.printCarriageReturn(f'Setting blur to {round(data[0],0)}%')

    def onDilate(self, data):
        self.dilateVal=int(data[0])
        self.dilateToggle=data[1]
        if self.dilateToggle:
            self.printCarriageReturn(f'Setting dilation to {round(data[0],0)}%')
            
    def checkCounts(self):
        self.printCarriageReturn(f'{(self.COUNTPARSE, self.COUNTADDVIDEO, self.COUNTCAPTURE)}')

    def onScaleBar(self):
        self.scaleBarOn+=1
    
    def onClear(self):
        clearVideos(self.videoLayout) # clear videos
        
        # Perform cleanup tasks
        self._running=False
        for connection in self.videoConnections:
            connection.signals.result.disconnect(self.onFrameReady)
            connection.stop()

        # Clean up video connections
        self.videoConnections.clear()
        self.threadPool.waitForDone()

        # create blank object
        tempVideo = QPushButton('')
        tempVideo.setObjectName('1_blank')
        tempVideo.setCheckable(True)
        tempVideo.setMaximumSize(self.dim[0], self.dim[1])
        tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
        tempVideo.setIconSize(QtCore.QSize(self.dim[0]*.99, self.dim[1]*.99))
        tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)

        self.numColumns = 2
        self.countPlaceholder=0
        self.numVideos=1
        self.videos.clear()
        self.emptyTruth = True
        self._running=True
        self.videoFiles.clear()
        self.videoLayout.addWidget(tempVideo, 0, 0)     
    
    def captureVideos(self, videoPath, videoName, resultCallback):
        cap = cv2.VideoCapture(videoPath)
        truth = True
        checked = False
        frameCount=0
        frames=[]
        evenFrame=None
        oddFrame=None
        temp=0
        detectedContours=None
        n=0
        
        if not cap.isOpened():
            msg = f"Error: Could not open video {videoName}.\n"
            self.printNewLine(msg)
            return   
        
        while self._running:
            ret, frame = cap.read()
            
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the first frame
                frameCount=0
                continue  # Continue the loop to read the video from the start

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # remove colors, most processes, such as frame differencing require this
            frame = cv2.resize(frame, (640, 480)) # resize to PyQt frame

            for video in self.videos:
                if video.objectName() == videoName:
                    checked = video.isChecked() # check if any video is checked/clicked on
            dark = False

            try:
                if checked or len(self.videos) == 0:
                    if self.contrastToggle.isChecked(): # contrast
                        frame = cv2.convertScaleAbs(frame, alpha=self.contrastVal, beta=0)
                        
                    if self.blurToggle: # gaussian blur
                        frame = cv2.GaussianBlur(frame, (self.blurVal, self.blurVal), 0)
                        
                    if self.dilateToggle: # dilation for emboss
                        frame = cv2.dilate(frame, None, iterations=self.dilateVal)
                        
                    if self.subBackToggle.isChecked(): # subtract background
                        if self.subBackMethod.currentText() == "MOG2": # Mixture of Gaussians 2 background subtraction method
                            frame = self.backSubsMOG2[videoName].apply(frame)
                        if self.subBackMethod.currentText() == "KNN": # K Nearest Neighbors background subtraction method
                            frame = self.backSubsKNN[videoName].apply(frame)
                            
                    if self.contourToggle.isChecked(): # emboss, misnamed as "contour" when originally writing, too lazy to replace every reference
                        kernel = np.array([[2, 1, 0],[1, 0, -1],[0, -1, -2]])
                        frame = cv2.convertScaleAbs(cv2.filter2D(frame, -1, kernel)*self.contourVal)
                        
                    if self.thresholdToggle.isChecked(): # threshold
                        _, frame = cv2.threshold(frame, self.thresholdVal, 255, cv2.THRESH_BINARY)
                        
                    elif len(self.videos) != 0:
                        dark = True
                    if any([self.contrastToggle.isChecked(),
                                                          self.blurToggle,
                                                          self.dilateToggle,
                                                          self.subBackToggle.isChecked(),
                                                          self.contourToggle.isChecked(),
                                                          self.thresholdToggle.isChecked(),
                                                          self.frameDiffToggle.isChecked()]): # don't darken if only one video is shown or if any process is toggled
                        dark = False

                    
                    frameCount+=1 # count frames
                    if self.resetFrames:
                        frames.clear()
                    else:
                        if frameCount % 2 == 0:
                            evenFrame=frame.copy()
                            temp=cap.get(cv2.CAP_PROP_POS_FRAMES)
                        elif cap.get(cv2.CAP_PROP_POS_FRAMES) > temp or frameCount == 1: # make sure that it takes the difference from a frame and a previous frame, not sure if necessary
                            oddFrame=frame.copy()
    
                        if evenFrame is not None and oddFrame is not None:
                            frames.append(cv2.absdiff(oddFrame, evenFrame)) # find difference between current 2 most recent frames
    
                        if len(frames) == self.consecutiveFrames:
                            if self.frameDiffToggle.isChecked():
                                try:
                                    detectedContours, points, pointObjects = self.frameDifference(frames, videoName, frameCount, n)
                                except Exception as e:
                                    self.printNewLine(f'Error detecting contours: {e}')
                                n+=1
                            frames.clear()
                        try:
                            if detectedContours is not None and self.frameDiffToggle.isChecked(): # have bounding boxes persist
                                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                                if self.bb:
                                    for contour in detectedContours:
                                        cv2.rectangle(frame, contour[0], contour[1], (0, 0, 255), thickness=2)
                                else:
                                    for point in points:
                                        cv2.circle(frame, point, radius=2, color=(0, 0, 255), thickness=-1)
                                    for pointObject in pointObjects:
                                        cv2.putText(frame, str(pointObject[0]), pointObject[1], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                                    self.processData()
                        except Exception as e:
                            self.printNewLine(f'Error drawing contours: {e}')
                            pass
                    
            except Exception as e:
                self.printNewLine(e)

            if self.scaleBarOn % 2 == 0: # add scale bar
                scaleBar(frame, 480, scaleFactor=12.6/2, scaleLength=20, divisions = 10, posX = 10, posY = 10, fontScale = 0.7, thickness = 1)

            if dark:
                frame = np.clip(frame*0.5, 0, 255).astype(np.uint8) # darken videos when selected
            pixmap = cv2FrameToPixmap(frame)  # convert cv2 frame to PyQt pixmap
            resultCallback.emit((pixmap, videoName)) # send frame from QRunnable thread to PyQt widget
            sleep(1/cap.get(cv2.CAP_PROP_FPS))  # adjust so that each frame is sent at the frame rate of the video, not just how fast it can process
            
            if truth:
                self.COUNTCAPTURE+=1
                # return upon successful first reading video, basically just a check that video was indeed read initially
                msg = f"Captured frame from {videoName} with dimensions: {frame.shape if frame is not None else 'None'}\n"
                self.printNewLine(msg)
                truth = False

        cap.release() # close video
        
    def returnVideos(self, layout):
        rows = layout.rowCount()
        cols = layout.columnCount()
        videos=[]
        for row in range(rows): 
            for col in range(cols):
                widget = layout.itemAtPosition(row, col)
                if widget is not None:
                    videos.append(widget.widget())
        return videos

    def onFrameReady(self, result):
        if result is not None:
            pixmap = result[0]
            name = result[1]
            videos = self.returnVideos(self.videoLayout)
            for video in videos:
                try:
                    if video.objectName() == name:
                        video.setIcon(QtGui.QIcon(pixmap))
                except Exception as e:
                    msg = f'No match for {video.objectName()}: {e}'
                    self.printNewLine(msg)
        else:
            pass
            
    def addVideos(self, name='BLANK', path=None): 
        if self.emptyTruth:
            self.videoLayout.itemAtPosition(0, 0).widget().setObjectName(f"{self.numVideos}_{name}")
            self.emptyTruth = False
        elif not self.numVideos == self.numColumns**2: # stop at max
            self.countPlaceholder+=1  
            self.numVideos+=1
            
            # Calculate the row and column for the widget (I have no idea why I need to add 1 but it works)
            row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
            column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
            if column == 1: # no clue why this works
                column = 2
                self.countPlaceholder+=1                
            
            tempVideo = QPushButton('')
            tempVideo.setObjectName(f"{self.numVideos}_{name}")
            tempVideo.setCheckable(True)
            tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
            tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            self.videoLayout.addWidget(tempVideo, row, column)  # Add to the layout
    
            rows = self.videoLayout.rowCount()
            cols = self.videoLayout.columnCount()
            width = self.dim[0]/rows
            height = self.dim[1]/(cols-1) # no clue
            self.videos = resizeVideos(rows, cols, width, height, self.videoLayout) # resize videos to fit
        else:
            self.countPlaceholder=0
            clearVideos(self.videoLayout)
            self.numColumns+=1
            self.numVideos+=1
            for video in self.videos:
                # Calculate the row and column for the widget
                row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
                column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
                if column == 1:
                    column = 2
                    self.countPlaceholder+=1   
                self.videoLayout.addWidget(video, row, column)  # Add to the layout
                self.countPlaceholder+=1

                        
            row = self.countPlaceholder // (self.numColumns + 1) # Integer division for row index
            column = self.countPlaceholder % (self.numColumns + 1) # Modulo operation for column index
            if column == 1:
                column = 2
                self.countPlaceholder+=1   
                
            tempVideo = QPushButton('')
            tempVideo.setObjectName(f"{self.numVideos}_{name}")
            tempVideo.setCheckable(True)
            tempVideo.setIcon(QtGui.QIcon(u"background.jpg"))
            tempVideo.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            self.videoLayout.addWidget(tempVideo, row, column)  # Add to the layout
            
            rows = self.videoLayout.rowCount()
            cols = self.videoLayout.columnCount()
            width = self.dim[0]/rows
            height = self.dim[1]/(cols-1) # no clue
            self.videos = resizeVideos(rows, cols, width, height, self.videoLayout) # resize videos to fit
        if path:
            self.data[f"{self.numVideos}_{name}"]={}
            self.dataTemp[f"{self.numVideos}_{name}"]={}
            self.COUNTADDVIDEO+=1
            self.printNewLine(f'Added video: {self.numVideos}_{name}')
            playVideo = VideoWorker(self.mutex, self.captureVideos, path, f'{self.numVideos}_{name}')
            playVideo.signals.result.connect(self.onFrameReady)
            self.videoConnections.append(playVideo)
            self.threadPool.start(playVideo)  # Start the worker
            self.backSubsMOG2[f'{self.numVideos}_{name}'] = cv2.createBackgroundSubtractorMOG2(history=450, varThreshold=16+48*self.subBackVal)
            self.backSubsKNN[f'{self.numVideos}_{name}'] = cv2.createBackgroundSubtractorKNN(dist2Threshold=100+500*self.subBackVal, history=500, detectShadows=True)
    
    def printNewLine(self, msg):
        # scroll to the bottom
        msg = str(msg)
        self.printOutput.append(msg)
        print(msg)
                
    def printCarriageReturn(self, msg):
        msg = str(msg)
        if self.printOutput.toPlainText() == '':
            self.printOutput.setPlainText(msg)
            return
        currentText = self.printOutput.toPlainText() # get current text
        lines = currentText.splitlines() # separate into lines at \n

        # overwite last message
        if msg[:5] not in lines[-1] != '': # prevent overwriting of previous line
            lines.append(msg)
        else:
            lines[-1] = msg

        # join lines back and update widget
        self.printOutput.setPlainText('\n'.join(lines))

        # scroll to the bottom
        self.printOutput.verticalScrollBar().setValue(self.printOutput.verticalScrollBar().maximum())

        print(msg.ljust(200), end='\r')
    
    def thresholdFunction(self, value):
        msg = f'Threshold: {value}'
        self.thresholdVal = (value/100)*255
        self.printCarriageReturn(msg) # print threshold value with \r

    def contrastFunction(self, value):
        msg = f'Contrast: {value}'
        self.printCarriageReturn(msg) # print contrast value with \r
        self.contrastVal = (value/100)+1 # values greater than one increase intensity, less than decrease

    def contourFunction(self, value):
        msg = f'Emboss: {value}'
        self.printCarriageReturn(msg) # print contour value wiht \r
        self.contourVal = (value/100)+1 # values greater than one increase intensity, less than decrease

    def onFrameDifferencing(self, *args): # allow for multiple types of parameters
        self.frameDifVal = self.frameDiffValue.value()
        msg = f'Frame Differencing with Sensitivity: {self.frameDifVal}%'
        if type(args[0]) == bool: # check for "toggled" state 
            if args[0]:
                self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on toggled
        else:
            self.printCarriageReturn(msg) # print frame differencing sensitivity with \r on value changed

    def onConsecutiveFramesValue(self):
        self.resetFrames = True
        self.consecutiveFrames = self.consecutiveFramesValue.value()
        QTimer.singleShot(500, self.onResetFrames)
        
    def onResetFrames(self):
        self.resetFrames = False
        self.printCarriageReturn(f'Frame Differencing with {self.consecutiveFrames} consecutive frames')
        
    def frameDifference(self, frames, name, frameCount, n):
        if self.resetFrames:
            self.printCarriageReturn('Waiting on frames. . .')
            return None
        else:
            rectangles=[]
            points=[]
            rectanglesCleaned=[]
            if not isinstance(frames, list):
                self.printNewLine(f'{frames} is not a list of frames!')
                return None
            else:
                # sum average difference over given set of frames, helps with smoothing out consistent motion
                sumFrames = cv2.convertScaleAbs(sum(frames))
                # find the contours around the white segmented areas
                contours, hierarchy = cv2.findContours(sumFrames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    # continue through the loop if contour area is less than some given sensitivity...
                    # ... helps in removing noise detection
                    if cv2.contourArea(contour) < 0+self.frameDifVal/10:
                        continue
                    # get the xmin, ymin, width, and height coordinates from the contours
                    (x, y, w, h) = cv2.boundingRect(contour)
                    # save the bounding boxes
                    rectangles.append([(x, y),(x+w, y+h)])
            
            # collect two boxes
            tempRectangles = rectangles[:]
            while tempRectangles:
                try:
                    rectA = tempRectangles.pop(0) # Take the first rectangle
                    merged = False # Flag to track if rectA has been merged
                except Exception as e:
                    self.printNewLine(e)

                while merged:  # Keep merging as long as there's an overlap
                    merged = False  # Reset the merged flag for this iteration
                
                    for rectB in tempRectangles[:]: # Use a copy of the list for iteration
                        try:
                            if self.isOverlapping(rectA, rectB):
                                rectanglesCleaned.append(self.mergeRects(rectA, rectB)) # Merge rectA with rectB
                                tempRectangles.remove(rectB) # Remove rectB from tempRectangles
                                merged = True # Mark that a merge happened
                        except Exception as e:
                            self.printNewLine(f'Error merging bounding boxes: {e}')
                            
                rectanglesCleaned.append(rectA)  # Add the possibly merged rectA to the cleaned list                
            
            for rectangle in rectanglesCleaned: # find points
                points.append(self.findCenter(rectangle))

            sensitivity=2

            pointObjects=[]
            if not self.record.isChecked():
                self.data[name]={}
                self.dataTemp[name]={}
            if self.record.isChecked():
                self.dataTemp[name][n]={}
                if len(self.data[name]) == 0: # assign initial points
                    self.printNewLine('assigning initial points')
                    i=0
                    for point in points: 
                        self.data[name][i]={f'frame_{frameCount}' : point}
                        self.dataTemp[name][n][i]={f'frame_{frameCount}' : point}
                        i+=1
                else:
                    oldPointObjects = list(self.dataTemp[name][n-1].keys())
                    j=max(oldPointObjects)
                    oldFrameCount = list(self.dataTemp[name][n-1][oldPointObjects[0]].keys())[0]
                    #print(f'iterating through {j} objects'.ljust(50), end='\r')
                    for newPoint in points:
                        match=False
                        for oldPointObject in oldPointObjects:
                            oldPoint = self.dataTemp[name][n-1][oldPointObject][oldFrameCount]
                            x1, y1 = oldPoint
                            x2, y2 = newPoint
                            distance = np.sqrt((x2 - x1)**2 + (y2-y1)**2)
                            if distance <= self.consecutiveFrames*self.persistence:
                                pointObjects.append([oldPointObject, newPoint])
                                self.data[name][oldPointObject].update({f'frame_{frameCount}' : newPoint})
                                self.dataTemp[name][n][oldPointObject]={f'frame_{frameCount}' : newPoint}
                                #self.printNewLine(f'found nearest neighbor in point {newPoint} with object {oldPointObject} : {oldPoint}')
                                match=True
                                break
                        if not match:
                            j+=1
                            pointObjects.append([j, newPoint])
                            self.data[name][j]={f'frame_{frameCount}' : newPoint}
                            self.dataTemp[name][n][j]={f'frame_{frameCount}' : newPoint}
                            
                        
            return rectanglesCleaned, points, pointObjects

    def mergeRects(self, rectA, rectB): # merge two rectangles
        x1 = min(rectA[0], rectB[0])
        y1 = min(rectA[1], rectB[1])
        x2 = max(rectA[2], rectB[2])
        y2 = max(rectA[3], rectB[3])
        return (x1, y1, x2, y2)  
    
    def findCenter(self, rect):
        (x1, y1), (x2, y2) = rect
        width = x2-x1
        height = y2-y1
        center = (int(x1+width/2), int(y1+height/2))
        return center
    
    def findArea(self, rect):
        (x1, y1), (x2, y2) = rect
        area = (x2-x1)*(y2-y1)
        return area
    
    def isOverlapping(self, rectA, rectB):
        (x1, y1), (x2, y2) = rectA # first box (top-left and bottom-right)
        (x3, y3), (x4, y4) = rectB # second box

        # check for overlap
        if x1 >= x4 or x3 >= x2:  # One box is completely to the left of the other
            return False
        if y1 >= y4 or y3 >= y2:  # One box is completely above the other
            return False  

        # If none of the above, the boxes overlap
        return True

    def subtractBackgroundFunction(self, *args):
        self.subBackVal = self.subBackValue.value()/100
        if self.subBackValue.value() != 0:
            QTimer.singleShot(250,self.onSubBack)

    def onSubBack(self):
        for video in self.returnVideos(self.videoLayout):
            try:
                if self.subBackMethod.currentText() == "MOG2":
                    self.backSubsMOG2[video.objectName()] = cv2.createBackgroundSubtractorMOG2(history=450, varThreshold=16+48*self.subBackVal)
                elif self.subBackMethod.currentText() == "KNN":
                    self.backSubsKNN[video.objectName()] = cv2.createBackgroundSubtractorKNN(dist2Threshold=100+500*self.subBackVal, history=450, detectShadows=True)
                self.printCarriageReturn(f'Subtracting background with intensity {self.subBackVal} using {self.subBackMethod.currentText()}')
            except Exception as e:
                msg=None
                self.printCarriageReturn(f'Failed to subtract background for {video.objectName()}: {e}')

    def openFile(self):
        options = QFileDialog.Options()
        files, _ = QFileDialog.getOpenFileNames(self, "Open Files", "", "Video Files (*.avi;*.mp4;*.mov;*.mkv;*.wmv;*.flv;*.mpeg;*.mpg)", options=options)
        
        if files:
            for file in files:
                self.parseFiles(file)

    def exportVideo(self):
        msg = 'Export :3'
        self.printNewLine(msg)
    
    def parseFiles(self, file):
        accepted = False
        fileType = file.split(".")[-1] # return file extension
        for videoType in videoFormats:
            if fileType == videoType:
                accepted = True # return true if file is a video
                break
        if not accepted:
            msg = f'File type: ".{fileType}" not accepted. \nAllowed file types:  ".avi," ".mp4", ".mov", ".mkv", ".wmv", ".flv", ".mpeg", ".mpg"\n'
            self.printNewLine(msg)
            return
        else:
            for videoFile in self.videoFiles:
                if file == videoFile:
                    msg = f'File "{file}" already added!'
                    self.printNewLine(msg)
                    accepted = False
                    return
        if accepted:
            msg = f'File opened: {file}'
            fileName = file.split(r'/')[-1]
            self.printNewLine(msg)
            try:
                self.addVideos(fileName, file)
                self.videoFiles.append(file)
                self.COUNTPARSE+=1
            except Exception as e:
                msg = f'Failed to add file: {file} {e}'
                self.printNewLine(msg)        
    
    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls(): # check if dragged item has file location/url
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasUrls():
            Qfiles = event.mimeData().urls()
            files = [Qfile.toLocalFile() for Qfile in Qfiles] # convert qt-type to file path string
            for file in files:
                self.parseFiles(file)
            event.accept()
        else:
            event.ignore()

    def processData(self):
        wires={}
        try:
            for i in data.loc(0):
                wire = int(i.name)
                wires[wire]={}
                positions=[]
                for j in i:
                    for key in j.keys():
                        positions.append([key.split('_')[1], j[key]])
                wires[wire].update(positions)
        except:
            pass
        
        fps = 30
        pixToum = 12.6
        tDif=[]
        pDif=[]
        velocity=[]
        for wire in wires:
            for i in range(1,len(list(wires[wire]))):
                frames = list(wires[wire])
                if len(frames) <= 2:
                    pass
                else:
                    x1, y1 = wires[wire][frames[i-1]]
                    x2, y2 = wires[wire][frames[i]]
                    p=float(np.sqrt((x2 - x1)**2 + (y2-y1)**2)/pixToum)
                    for i in range(0, len(frames)):
                        frames[i]=int(frames[i])
                    frames.sort()
                    oldTime = frames[i-1]
                    newTime = frames[i]
                    t=float(newTime - oldTime)/30
                    if t == 0:
                        pass
                    else:
                        velocity.append(round(p/t,4))
                        tDif.append(t)
                        pDif.append(p)
        print(f'Average velocity: {round(np.average(velocity),4)} um/s', end='\r')
    
    def closeEvent(self, event):
        try:
            self._running=False
            # Perform cleanup tasks
            for connection in self.videoConnections:
                connection.signals.result.disconnect(self.onFrameReady)
                connection.stop()

            # Clean up video connections
            self.videoConnections.clear()
            self.threadPool.waitForDone()

            # Perform any other necessary cleanup tasks
            event.accept()  # Accept the event to close the window
            
            super().closeEvent(event)
        except Exception as e:
            msg = f"Error during close event: {e}"
            self.printNewLine(msg)
            event.ignore()  # Optionally ignore the event if cleanup fails
        print('\nExited')

if not QtWidgets.QApplication.instance():
    app = QtWidgets.QApplication(sys.argv)
else:
    app = QtWidgets.QApplication.instance()

if __name__ == '__main__':
    window = MainWindow()
    app.setStyle('Windows')
    window.show()
    print('Running\n')
    app.exec()

Running

File opened: D:/NPM/09-19-2024/DC/5V 30 sec_original.avi
Added video: 1_5V 30 sec_original.avi
Captured frame from 1_5V 30 sec_original.avi with dimensions: (480, 640)



C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\lib\_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Empty DataFramey: nan um/s
Columns: [1_5V 30 sec_original.avi]
Index: []


C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\lib\_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


assigning initial points/s
                              1_5V 30 sec_original.avi
0    {'frame_241': (595, 478), 'frame_253': (571, 4...
1                            {'frame_241': (557, 475)}
2    {'frame_241': (88, 475), 'frame_253': (105, 45...
3    {'frame_241': (529, 472), 'frame_253': (476, 4...
4    {'frame_241': (299, 475), 'frame_253': (303, 4...
..                                                 ...
234                          {'frame_337': (590, 213)}
235                          {'frame_337': (219, 216)}
236                          {'frame_337': (633, 207)}
237                           {'frame_337': (88, 131)}
238                           {'frame_337': (83, 126)}

[239 rows x 1 columns]
Empty DataFramey: 7.2406 um/ssecutive frames                                                                                                                                                            
Columns: [1_5V 30 sec_original.avi]
Index: []
assigning initial points/s


...

assigning initial points/s
                              1_5V 30 sec_original.avi                                                                                                                                                  
0    {'frame_350': (592, 477), 'frame_365': (573, 4...
1    {'frame_350': (501, 463), 'frame_365': (504, 4...
2    {'frame_350': (606, 453), 'frame_365': (630, 4...
3                            {'frame_350': (380, 448)}
4                            {'frame_350': (637, 443)}
..                                                 ...
137  {'frame_635': (338, 455), 'frame_650': (340, 4...
138                          {'frame_635': (631, 446)}
139                          {'frame_635': (604, 275)}
140                          {'frame_635': (615, 269)}
141                          {'frame_650': (458, 372)}

[142 rows x 1 columns]
Empty DataFramey: 4.5531 um/s
Columns: [1_5V 30 sec_original.avi]
Index: []
assigning initial points/s


C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\lib\_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


                             1_5V 30 sec_original.avi
0   {'frame_710': (532, 478), 'frame_725': (587, 4...
1   {'frame_710': (587, 475), 'frame_725': (609, 4...
2   {'frame_710': (465, 465), 'frame_725': (413, 4...
3                           {'frame_710': (412, 441)}
4                           {'frame_710': (476, 439)}
..                                                ...
92                          {'frame_920': (477, 208)}
93                          {'frame_935': (266, 289)}
94  {'frame_935': (517, 101), 'frame_950': (480, 59)}
95                           {'frame_950': (70, 477)}
96                          {'frame_950': (501, 173)}

[97 rows x 1 columns]
Empty DataFramey: 3.3434 um/s
Columns: [1_5V 30 sec_original.avi]
Index: []


C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\lib\_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\Nolan\anaconda3\envs\npm\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


assigning initial points/s
                              1_5V 30 sec_original.avi
0    {'frame_69': (431, 474), 'frame_84': (449, 431...
1    {'frame_69': (560, 469), 'frame_84': (549, 418...
2    {'frame_69': (411, 472), 'frame_84': (366, 426...
3    {'frame_69': (615, 465), 'frame_84': (585, 405...
4                             {'frame_69': (626, 466)}
..                                                 ...
140  {'frame_264': (540, 369), 'frame_279': (572, 3...
141                          {'frame_279': (634, 472)}
142                          {'frame_279': (589, 229)}
143                          {'frame_279': (598, 221)}
144                          {'frame_279': (586, 205)}

[145 rows x 1 columns]
STOPage velocity: 4.9148 um/s

Exited


Average velocity: 4.8861 um/s
